# Homework 2

In [166]:
import re
import time
import requests
import numpy as np
import pandas as pd
from scrapy.http import TextResponse

## Problem 1

In [34]:
class Quotes:
  
    base_url = "http://quotes.toscrape.com"

    def __init__(self, url):
    self.url = url
    self.get_response()

    def get_response(self):
    page = requests.get(self.url)
    if page.status_code == 200:
      self.response = TextResponse(body=page.text,url=page.url,encoding="utf-8")
    else:
      self.response = None
    return self.response

    def get_data(self):
    if self.response:
        return {
          "quote":self.response.css("span.text::text").extract(),
          "author":self.response.css("small.author::text").extract(),
          "author_page":[self.base_url+i for i in self.response.css("small.author ~ a::attr(href)").extract()],
          "tags":[i.css("a.tag::text").extract() for i in self.response.css("div.tags")]
      }

    def get_next_pagelink(self):
    if self.response:
        next_button = self.response.css("li.next > a::attr(href)").extract_first()
        if next_button:
        return self.base_url + next_button
        else:
        return None

In [ ]:
url = "http://quotes.toscrape.com/page/1/"
SLEEP = 3
PAGES = []
while True:
  page = Quotes(url)
  PAGES.append(pd.DataFrame(page.get_data()))
  next_page = page.get_next_pagelink()
  print(f"Finished scraping page {url.split('/')[-2]}")
  if next_page:
    url = next_page
    time.sleep(SLEEP)
  else:
    print("No more page found, stoping scraper")
    break

data = pd.concat(PAGES)
assert data.shape == (100,4)
data.to_csv("all_quotes.csv",index=False)

## Problem 2

In [144]:
class Movies:
  
  def __init__(self, url):
    self.url = url
    self.get_response()

  def get_response(self):
    page = requests.get(self.url)
    if page.status_code == 200:
      self.response = TextResponse(body=page.text,url=page.url,encoding="utf-8")
    else:
      self.response = None
    return self.response

  def get_data(self):
    return {
        "title": self.response.css("td.titleColumn > a::text").extract(),
        "year": self.response.css("td.titleColumn > span.secondaryInfo::text").re("\((\d{4})\)"),
        "ranking": self.response.xpath("//td[@class='titleColumn']/div[@class='velocity']/text()[1]").re("\d+"),
        "rating": [i.css("strong::text").extract_first() for i in self.response.css("td[class*='imdbRating']")],
        "hyperlink": self.response.css("td.titleColumn > a::attr(href)").extract()
    }

In [153]:
movies = Movies("https://www.imdb.com/chart/moviemeter/")
data = movies.get_data()
assert np.var([len(i) for i in data.values()])==0
data = pd.DataFrame(data)
assert data.shape==(100,5)
data.to_csv("top_movies.csv",index=False)

## Problem 3

In [178]:
class Books:
  
  base_url = "http://books.toscrape.com/catalogue/"

  def __init__(self, url):
    self.url = url
    self.get_response()

  def get_response(self):
    page = requests.get(self.url)
    if page.status_code == 200:
      self.response = TextResponse(body=page.text,url=page.url,encoding="utf-8")
    else:
      self.response = None
    return self.response

  def get_data(self):
    return {
        "title": self.response.css("a[title]::attr(title)").extract(),
        "price": self.response.css("p.price_color::text").re("\d+\.\d+"),
        "rating": [i.split(" ")[-1] for i in self.response.css("p[class^='star-rating']::attr(class)").extract()],
        "availability": self.response.xpath("//p[contains(@class,'stock')]/text()[2]").re("\w+.+\w"),
        "hyperlink": [self.base_url+i for i in self.response.css("a[title]::attr(href)").extract()]
        }

  def get_next_pagelink(self):
    if self.response:
      next_button = self.response.css("li.next > a::attr(href)").extract_first()
      if next_button:
        return self.base_url + next_button
      else:
        return None

In [ ]:
url = "http://books.toscrape.com/catalogue/page-1.html"
SLEEP = 3
PAGES = []
while True:
  page = Books(url)
  PAGES.append(pd.DataFrame(page.get_data()))
  next_page = page.get_next_pagelink()
  page_num = re.findall('\d+',url)[0]
  print(f"Finished scraping page {page_num}")
  if next_page:
    url = next_page
    time.sleep(SLEEP)
  else:
    print("No more page found, stoping scraper")
    break

data = pd.concat(PAGES)
assert data.shape == (20*50,5)
data.to_csv("all_books.csv",index=False)